In [2]:
!pip install catboost

     |████████████████████████████████| 64.8MB 65kB/s 


In [3]:
!pip install unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


In [4]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import re
import unidecode
import nltk
from nltk.tokenize import TweetTokenizer
import string
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
import category_encoders as ce

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
id='1cQhZ_2NeVAXqkXpmaCQbJVxLP4b1lNAf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [8]:
id='12nWwwnj9jbtAsk0VENlQaV3d6jclQ-1S'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [9]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [11]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [12]:
train["clean_text"] = train["text"].apply(lambda x: cleaning_text(x))
test["clean_text"] = test["text"].apply(lambda x: cleaning_text(x))

In [13]:
train["keyword"] = train["keyword"].str.replace('%20',' ')
train["keyword"] = train["keyword"].astype('category')
test["keyword"] = test["keyword"].str.replace('%20',' ')
test["keyword"] = test["keyword"].astype('category')

In [14]:
train.rename(columns={"target":"target_label"}, inplace=True)
train.rename(columns={"location":"location_original"}, inplace=True)
train.rename(columns={"id":"id_original"}, inplace=True)
train.rename(columns={"text":"text_original"}, inplace=True)
train.rename(columns={"keyword":"keyword_original"}, inplace=True)
train.head()

,id_original,keyword_original,location_original,text_original,target_label,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [15]:
test.rename(columns={"target":"target_label"}, inplace=True)
test.rename(columns={"location":"location_original"}, inplace=True)
test.rename(columns={"id":"id_original"}, inplace=True)
test.rename(columns={"text":"text_original"}, inplace=True)
test.rename(columns={"keyword":"keyword_original"}, inplace=True)
test.head()

,id_original,keyword_original,location_original,text_original,clean_text
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different cities stay safe ev...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan


In [16]:
binary_encoder = ce.BinaryEncoder(cols=['keyword_original'])
binary = binary_encoder.fit_transform(train['keyword_original'])
train_bin = pd.concat([train, binary], axis=1)
train_bin.head()

,id_original,keyword_original,location_original,text_original,target_label,clean_text,keyword_original_0,keyword_original_1,keyword_original_2,keyword_original_3,keyword_original_4,keyword_original_5,keyword_original_6,keyword_original_7,keyword_original_8
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us,0,0,0,0,0,0,0,0,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,0,0,0,0,0,0,0,0,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...,0,0,0,0,0,0,0,0,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...,0,0,0,0,0,0,0,0,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...,0,0,0,0,0,0,0,0,1


In [ ]:
# binary_encoder = ce.BinaryEncoder(cols=['keyword_original'])
# binary = binary_encoder.fit_transform(test['keyword_original'])
# test_bin = pd.concat([test, binary], axis=1)
# test_bin.head()

In [17]:
#BOW
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(train["clean_text"])

vec_train = vectorizer.fit_transform(train["clean_text"])
feature_words = vectorizer.get_feature_names()
print(len(feature_words))
df_bow_train = pd.DataFrame(vec_train.toarray(), columns=feature_words)
df_bow_train.head()

14190


,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
vec_test = vectorizer.fit_transform(test["clean_text"])
feature_words = vectorizer.get_feature_names()
print(len(feature_words))
df_bow_test = pd.DataFrame(vec_test.toarray(), columns=feature_words)
df_bow_test.head()

8674


,aa,aaaaaa,aabn,aapatwork,aaron,aba,abandoned,abandons,abba,abbog,abbott,abbswinston,abc,abcnews,abdulrasheed,abe,abella,abh,abide,ability,ablaze,able,abnormally,abolishing,abomb,abombs,abomination,abooortiooonnns,aboriginal,aborted,abortions,abounds,abrianna,abs,absol,absolute,absolutely,abstorm,abt,abuse,...,youcantsitwithus,youd,yougslavia,youku,youll,young,youre,yoursstand,youth,youtube,youve,ypg,yr,yrs,yup,yyc,yychail,yycstorm,yyctraffic,yycweather,zack,zayn,zealand,zero,zeroedgea,zeroes,zerohedge,zion,zionist,zippers,zirngast,zix,zombie,zombies,zone,zones,zouis,zouma,zuma,zx
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
print(df_bow_train.shape)
print(df_bow_test.shape)

(7613, 14190)
(3263, 8674)


In [19]:
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>1)]
train_final = pd.concat([train_bin,filter_train], axis="columns")
print(train_final.shape)
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>=5)]
train_final_2 = pd.concat([train_bin,filter_train], axis="columns")
print(train_final_2.shape)
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>=10)]
train_final_3 = pd.concat([train_bin,filter_train], axis="columns")
print(train_final_3.shape)

(7613, 6064)
(7613, 2506)
(7613, 1338)


In [ ]:
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>1)]
# test_final = pd.concat([test_bin,filter_test], axis="columns")
# print(test_final.shape)
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>=5)]
# test_final_2 = pd.concat([test_bin,filter_test], axis="columns")
# print(test_final_2.shape)
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>=10)]
# test_final_3 = pd.concat([test_bin,filter_test], axis="columns")
# print(test_final_3.shape)

In [20]:
X = train_final_2.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final_2["target_label"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 2500)
(5709,)
(1904, 2500)
(1904,)


In [22]:
# feature selection
def select_features(X_train, y_train, X_test, function, k):
    fs = SelectKBest(score_func=function, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

## XGBoost

In [31]:
# seleccionando las 100 features mas relevantes
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 100)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 100)

In [42]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=7, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [43]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.744748


In [44]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=7, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [45]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.664916


In [46]:
# 300 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 300)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 300)

In [57]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [58]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.780462


In [62]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [63]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.699055


In [64]:
# 500 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 500)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 500)

In [71]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [72]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794118


In [73]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [74]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.717962


In [75]:
# 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [84]:
model = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [85]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.803571


In [86]:
model = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [87]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.759454


In [88]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.792606


In [89]:
X = train_final_3.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final_3["target_label"]

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 1332)
(5709,)
(1904, 1332)
(1904,)


In [91]:
# 100 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 100)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 100)

In [92]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=7, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [93]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.744748


In [94]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=7, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [95]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.675420


In [96]:
# 300 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 300)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 300)

In [97]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [98]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.777311


In [99]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [100]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.734244


In [101]:
# 500 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 500)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 500)

In [102]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [103]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794118


In [104]:
model = xgb.XGBClassifier(n_estimators=200, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [105]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.765756


In [106]:
# 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [107]:
model = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [108]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.797269


In [109]:
model = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [110]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.782038


In [111]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.782798


## LightGBM



In [113]:
X = train_final_2.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final_2["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 2500)
(5709,)
(1904, 2500)
(1904,)


In [114]:
#con 100 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 100)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 100)

In [115]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=4, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=4,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [116]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.745273


In [117]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=4, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=4,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [118]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.633403


In [119]:
#con 300 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 300)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 300)

In [142]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [143]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.771008


In [144]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [145]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.667542


In [146]:
#con 500 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 500)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 500)

In [169]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=350, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=350, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [170]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.789391


In [171]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=350, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=350, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [172]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.738971


In [173]:
#con 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [208]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [209]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.804622


In [206]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [207]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.751050


In [210]:
kfold = KFold(n_splits=8)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.787528


In [211]:
X = train_final.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 6058)
(5709,)
(1904, 6058)
(1904,)


In [212]:
#con 300 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 300)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 300)

In [213]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [214]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.771008


In [215]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=250, min_child_samples=20, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [216]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.655987


In [217]:
#con 500 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 500)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 500)

In [218]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=350, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=350, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [219]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.789391


In [220]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=350, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=350, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [221]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.673845


In [222]:
#con 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [223]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [224]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.800420


In [225]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs2, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [226]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.714286


In [228]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.786300


## CatBoost

In [24]:
X = train_final_2.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final_2["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 2500)
(5709,)
(1904, 2500)
(1904,)


In [25]:
#con 100 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 100)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 100)

In [38]:
model = cb.CatBoostClassifier(iterations=200, depth=11, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [39]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.742122


In [40]:
model = cb.CatBoostClassifier(iterations=200, depth=11, learning_rate=0.1)
model.fit(X_train_fs2, y_train, verbose=False)

In [41]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.632353


In [42]:
#con 300 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 300)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 300)

In [60]:
model = cb.CatBoostClassifier(iterations=200, depth=11, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [61]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.784139


In [62]:
model = cb.CatBoostClassifier(iterations=200, depth=11, learning_rate=0.1)
model.fit(X_train_fs2, y_train, verbose=False)

In [63]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.707458


In [64]:
#con 500 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 500)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 500)

In [81]:
model = cb.CatBoostClassifier(iterations=300, depth=9, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [82]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.793067


In [83]:
model = cb.CatBoostClassifier(iterations=200, depth=11, learning_rate=0.1)
model.fit(X_train_fs2, y_train, verbose=False)

In [84]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.727416


In [85]:
#con 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [106]:
model = cb.CatBoostClassifier(iterations=250, depth=9, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [107]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.803571


In [108]:
model = cb.CatBoostClassifier(iterations=250, depth=9, learning_rate=0.1)
model.fit(X_train_fs2, y_train, verbose=False)

In [109]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.778887


In [110]:
X = train_final.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 6058)
(5709,)
(1904, 6058)
(1904,)


In [111]:
#con 1000 features
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, chi2, 1000)
X_train_fs2, X_test_fs2, fs2 = select_features(X_train, y_train, X_test, mutual_info_classif, 1000)

In [134]:
model = cb.CatBoostClassifier(iterations=300, depth=7, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [135]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.798319


In [114]:
model = cb.CatBoostClassifier(iterations=250, depth=9, learning_rate=0.1)
model.fit(X_train_fs2, y_train, verbose=False)

In [115]:
y_test_hat = model.predict(X_test_fs2)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.731092


In [136]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

0:	learn: 0.6831395	total: 9.16ms	remaining: 2.74s
1:	learn: 0.6737147	total: 18.5ms	remaining: 2.76s
2:	learn: 0.6670245	total: 26.7ms	remaining: 2.64s
3:	learn: 0.6592938	total: 35.4ms	remaining: 2.62s
4:	learn: 0.6540839	total: 45.6ms	remaining: 2.69s
5:	learn: 0.6496927	total: 54.2ms	remaining: 2.66s
6:	learn: 0.6456821	total: 62.5ms	remaining: 2.62s
7:	learn: 0.6424131	total: 70.6ms	remaining: 2.58s
8:	learn: 0.6388572	total: 78.6ms	remaining: 2.54s
9:	learn: 0.6343544	total: 87.2ms	remaining: 2.53s
10:	learn: 0.6311945	total: 95.3ms	remaining: 2.5s
11:	learn: 0.6273949	total: 104ms	remaining: 2.49s
12:	learn: 0.6237709	total: 112ms	remaining: 2.48s
13:	learn: 0.6207113	total: 121ms	remaining: 2.46s
14:	learn: 0.6174506	total: 129ms	remaining: 2.45s
15:	learn: 0.6133254	total: 137ms	remaining: 2.44s
16:	learn: 0.6107726	total: 146ms	remaining: 2.43s
17:	learn: 0.6082913	total: 154ms	remaining: 2.42s
18:	learn: 0.6049112	total: 163ms	remaining: 2.4s
19:	learn: 0.6020700	total: 171m